# Desafio 02
#### _bootcamp_ da Escola Supercomputador Santos Dumont - 2025
por Calebe de Paula Bianchini

### Mais um problema de com matrizes: Jacobi!

O método iterativo de Jacobi é uma maneira de calcular iterativamente a solução de uma equação diferencial, refinando continuamente a solução até que a resposta tenha convergido para um estado estável ou até que um número fixo de etapas tenha sido concluído, e a resposta seja considerada suficientemente precisa ou não tenha convergido.  

O código de exemplo a seguir representa um plano 2D de material que foi dividido em uma grade de células de tamanho igual. À medida que o calor é aplicado ao centro desse plano, a equação de Laplace determina como o calor se transferirá de um ponto da grade para outro ao longo do tempo. Para calcular a temperatura de um determinado ponto da grade na próxima iteração temporal, basta calcular a média das temperaturas dos pontos vizinhos da iteração atual.  

Uma vez que o próximo valor para cada ponto da grade é calculado, esses valores tornam-se a temperatura atual, e o cálculo continua. A cada etapa, a variação máxima de temperatura entre todos os pontos da grade determinará se o problema convergiu para um estado estacionário.

![Jacobi](./img/mpi_jacobi.png)

In [ ]:
%%writefile mpi_jacobi.c

/*
    This program solves Laplace's equation on a regular 2D grid using simple Jacobi iteration.

    The stencil calculation stops when  ( err >= CONV_THRESHOLD OR  iter > ITER_MAX )
*/
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <sys/time.h>

#define CONV_THRESHOLD 1.0e-5f // threshold of convergence

// matrix to be solved
double **grid;

// auxiliary matrix
double **new_grid;

// size of each side of the grid
int size;
int iter_max_num;

// return the absolute value of a number
double absolute(double num)
{
    if (num < 0)
        return -1.0 * num;
    return num;
}

// allocate memory for the grid
void allocate_memory()
{
    grid = (double **)malloc(size * sizeof(double *));
    new_grid = (double **)malloc(size * sizeof(double *));

    for (int i = 0; i < size; i++)
    {
        grid[i] = (double *)malloc(size * sizeof(double));
        new_grid[i] = (double *)malloc(size * sizeof(double));
    }
}

// initialize the grid
void initialize_grid()
{
    int center = size / 2;
    int linf = center - (size / 10);
    int lsup = center + (size / 10);
    for (int i = 0; i < size; i++)
    {
        for (int j = 0; j < size; j++)
        {
            // inicializa região de calor no centro do grid
            if (i >= linf && i <= lsup && j >= linf && j <= lsup)
                grid[i][j] = 100;
            else
                grid[i][j] = 0;
            new_grid[i][j] = 0.0;
        }
    }
}

// save the grid in a file
void save_grid()
{

    for (int i = 0; i < size; i++)
    {
        for (int j = 0; j < size; j++)
        {
            fprintf(stdout, "%lf ", grid[i][j]);
        }
        fprintf(stdout, "\n");
    }
}

int main(int argc, char *argv[])
{

    size = 2048;
    iter_max_num = 15000;
    // retire o comentário abaixo para fazer a leitura do valor
    // fscanf(stdin, "%d", &size);
    // fscanf(stdin, "%d", &iter_max_num);

    int hasError = 1;
    int iter = 0;

    struct timeval time_start;
    struct timeval time_end;

    // allocate memory to the grid (matrix)
    allocate_memory();

    // set grid initial conditions
    initialize_grid();

    // Jacobi iteration
    // This loop will end if either the maximum change reaches below a set threshold (convergence)
    // or a fixed number of maximum iterations have completed
    while (hasError && iter < iter_max_num)
    {

        hasError = 0;
        // calculates the Laplace equation to determine each cell's next value
        for (int i = 1; i < size - 1; i++)
        {
            for (int j = 1; j < size - 1; j++)
            {

                new_grid[i][j] = 0.25 * (grid[i][j + 1] + grid[i][j - 1] +
                                         grid[i - 1][j] + grid[i + 1][j]);

                double errorCurrent = absolute(new_grid[i][j] - grid[i][j]);
                if (errorCurrent > CONV_THRESHOLD)
                {
                    hasError = 1;
                }
            }
        }
        // copie the next values into the working array for the next iteration
        for (int i = 1; i < size - 1; i++)
        {
            for (int j = 1; j < size - 1; j++)
            {
                grid[i][j] = new_grid[i][j];
            }
        }
        iter++;
    }


    // save the final grid in file
    save_grid();

    return 0;
}

### Você lembra como compilar esse código?

In [ ]:
!gcc ...

### Você lembra como executá-lo? E, medir tempo? E, alterar a quantidade de processos?

In [ ]:
!time -p mpirun ....

### Perguntas para pensar

* o resultado para 1 ou mais processos se manteve o mesmo?
* o tempo de execução mudou proporcionalmente ao aumentar a quantidade de processos?
* e se alterarmos os valores de entrada (_size_ ou _itern..._), há mudança no que estamos fazendo/pensado em paralelimos?
* é possível perceber o impacto da comunicação entre os processos na sua solução?

Maiores detalhes sobre Programação Paralela e MPI, vejam:

* __Programação Paralela e Distribuída__ _com MPI, OpenMP e OpenACC para computação de alto desempenho_, em [Casa do Código](https://www.casadocodigo.com.br/products/livro-programacao-paralela).

